In [2]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import requests
import re

In [3]:
url = "https://goodreads.com/book/show/1001053.Between_Two_Fires"
authors_desc = {'link':[], 'Description':[]}
df = pd.read_csv('../csv/goodreads_clean.csv').dropna()

In [4]:
df

,author,bookformat,desc,genre,img,isbn,link,pages,rating,reviews,title,totalratings
0,Laurence M. Hauptman,Hardcover,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,https://goodreads.com/book/show/1001053.Betwee...,0,3.52,5,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Paperback,Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,https://goodreads.com/book/show/10010552-fashi...,576,4.51,6,Fashion Sourcebook 1920s,41
2,Andy Anderson,Paperback,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,https://goodreads.com/book/show/1001077.Hungar...,124,4.15,2,Hungary 56,26
3,Carlotta R. Anderson,Hardcover,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,https://goodreads.com/book/show/1001079.All_Am...,324,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,6
4,Jeffrey Pfeffer,Hardcover,Why is common sense so uncommon when it comes ...,"Business,Leadership,Romance,Historical Romance...",https://i.gr-assets.com/images/S/compressed.ph...,875848419,https://goodreads.com/book/show/1001090.The_Hu...,368,3.73,7,The Human Equation: Building Profits by Puttin...,119
...,...,...,...,...,...,...,...,...,...,...,...,...
68690,Philip Hoare,Paperback,"A startling new book, his most personal to dat...","Nonfiction,Environment,Nature,Travel,Science,A...",https://i.gr-assets.com/images/S/compressed.ph...,7412118,https://goodreads.com/book/show/17320067-the-s...,350,3.77,74,The Sea Inside,497
68691,Sarah Lean,Paperback,Sometimes when things are broken you can't fix...,"Animals,Childrens,Middle Grade,Horses,Fiction,...",https://i.gr-assets.com/images/S/compressed.ph...,7455054,https://goodreads.com/book/show/17320079-a-hor...,288,3.97,94,A Horse for Angel,535
68692,C. John Miller,Paperback,In these warm reflections on his own growth as...,"Christianity,Evangelism,Christian,Religion,The...",https://i.gr-assets.com/images/S/compressed.ph...,875523919,https://goodreads.com/book/show/173203.A_Faith...,160,4.27,20,A Faith Worth Sharing: A Lifetime of Conversat...,122
68693,Albert Marrin,Hardcover,"John Brown is a man of many legacies, from her...","Nonfiction,History,Biography,Military History,...",https://i.gr-assets.com/images/S/compressed.ph...,307981533,https://goodreads.com/book/show/17320669-a-vol...,256,3.63,51,A Volcano Beneath the Snow: John Brown's War A...,156


In [5]:
authors = []
links = []
for index, row in df.iterrows():
    x_list = row['author'].split(",")
    for x in x_list:
        if x not in authors:
            authors.append(x)
            links.append(row['link'])
len(authors)

62593

In [6]:
new_df = pd.DataFrame(data = {'link': links, 'author': authors})

In [8]:
counter = 0
def getAuthorDescription(link):
    global counter
    try:
        #print('Getting reviews from ' + str(link))
        print(counter)
        r = requests.get(str(link))
        soup = BeautifulSoup(r.text, 'html.parser')
        
        req = soup.find('div', {'class':'bookAuthorProfile__about'})

        #req2 = soup.find('div', {'class':'bookAuthorProfile__name'})
        #name = req2.select('a')[0]
        #name = re.sub("href=*\"", "", str(name))
        #name = re.sub(r'<.*?>', '', str(name))
        #name = re.sub("\n", "", str(name))
    
        if req == None or not req: 
            authors_desc['link'].append(link)
            authors_desc['Description'].append("")
            return

        auth = req.select('span', {'id':'freeText'})
        
        if len(auth) == 2:
            auth = auth[1]
        else:
            auth = auth[0]

        auth = re.sub(r'<.*?>', '', str(auth))
        authors_desc['link'].append(link)
        authors_desc['Description'].append(str(auth))

        counter += 1
    except Exception as e:
        print(str(e) + " -> " + url)
    return True

In [9]:
new_df['link'].apply(getAuthorDescription)

0
0
0
0
0
0
0
0
0
0
1
1
2
2
3
3
4
4
4
5
6
7
8
9
9
9
9
10
10
10
10
10


KeyboardInterrupt: 

In [ ]:
df1 = pd.DataFrame(authors_desc, columns = ['link', 'Description'])
df1.to_csv('authors.csv', encoding='utf-8', index=False)